In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
train = pd.read_csv("../data/schemas/warm_up/TrainSet.csv", sep=",")
test = pd.read_csv("../data/schemas/warm_up/TestSet.csv", sep=",")

In [6]:
NUM_OF_TIMESTEPS_INPUT = 48
NUM_OF_TIMESTEPS_OUTPUT = 24

In [7]:
threshold = 0
column_to_predict = "kg_CO2/kWh"

independent_variables = []


for column in train:
    if abs(train[column].corr(train[column_to_predict])) > threshold:
        independent_variables.append(column)

independent_variables.remove(column_to_predict)
independent_variables.remove("Index")
independent_variables

['Hour_1',
 'Day Type_1',
 'Indoor Temperature (C)_1',
 'Average Unmet Cooling Setpoint Difference (C)_1',
 'Indoor Relative Humidity (%)_1',
 'Equipment Electric Power (kWh)_1',
 'DHW Heating (kWh)_1',
 'Cooling Load (kWh)_1',
 'Solar Generation (W/kW)_1',
 'Occupant Count (people)_1',
 'Temperature Set Point (C)_1',
 'Hour_2',
 'Day Type_2',
 'Indoor Temperature (C)_2',
 'Average Unmet Cooling Setpoint Difference (C)_2',
 'Indoor Relative Humidity (%)_2',
 'Equipment Electric Power (kWh)_2',
 'DHW Heating (kWh)_2',
 'Cooling Load (kWh)_2',
 'Solar Generation (W/kW)_2',
 'Occupant Count (people)_2',
 'Temperature Set Point (C)_2',
 'Hour_3',
 'Day Type_3',
 'Indoor Temperature (C)_3',
 'Average Unmet Cooling Setpoint Difference (C)_3',
 'Indoor Relative Humidity (%)_3',
 'Equipment Electric Power (kWh)_3',
 'DHW Heating (kWh)_3',
 'Cooling Load (kWh)_3',
 'Solar Generation (W/kW)_3',
 'Occupant Count (people)_3',
 'Temperature Set Point (C)_3',
 'Outdoor Drybulb Temperature (C)',
 'Ou

In [8]:
X_train = train[independent_variables]
# X_train = train[["Hour_1", "Day Type_1"]]
Y_train = train[column_to_predict]

In [9]:
X_train.shape

(648, 50)

In [10]:
def create_sequences(input_data, output_data, timesteps_input, timesteps_output):
    sequences, targets = [], []

    for i in range(len(input_data) - timesteps_input - timesteps_output + 1):
        seq = input_data[i:i + timesteps_input]
        #print(f"i = {i}, seq = {seq}")
        target = output_data[i + timesteps_input: i + timesteps_input + timesteps_output]
        #print(f"i = {i}, target = {target}")
        sequences.append(seq)
        targets.append(target)

    return np.array(sequences), np.array(targets)

In [11]:
input_sequences, target_variables = create_sequences(X_train, Y_train, NUM_OF_TIMESTEPS_INPUT, NUM_OF_TIMESTEPS_OUTPUT)
print(input_sequences[0])
print(target_variables[0])
print(input_sequences.shape)
print(target_variables.shape)

[[0.         0.66666667 0.42905032 ... 0.90353129 0.         0.        ]
 [0.04347826 0.66666667 0.30943362 ... 0.73034217 0.         0.        ]
 [0.08695652 0.66666667 0.30781619 ... 0.87445669 0.         0.        ]
 ...
 [0.91304348 0.83333333 0.29064017 ... 0.29065842 0.         0.        ]
 [0.95652174 0.83333333 0.04895782 ... 0.59164279 0.         0.        ]
 [1.         0.83333333 0.00208331 ... 0.53563308 0.         0.        ]]
[0.59739428 0.49429981 0.38708765 0.28880037 0.2346659  0.22880156
 0.26241434 0.31385395 0.36274065 0.32912399 0.41859464 0.59690114
 0.74685743 0.82796725 0.84589733 0.83725959 0.82125879 0.80709442
 0.78806505 0.73835998 0.71748058 0.73547528 0.69513525 0.71729728]
(577, 48, 50)
(577, 24)


In [12]:
# importing libraries
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Dropout
from keras.optimizers import SGD

In [13]:
print((X_train.shape[1],1))

(50, 1)


In [16]:
# initializing the sequential RNN
model = keras.Sequential([
        keras.layers.Dense(20, activation='tanh', input_shape=(X_train.shape[1],1)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(20, activation='tanh'),
        keras.layers.Dense(10, activation='tanh'),
        keras.layers.Dense(1),
    ]) #amount of layers & neurons to be adjusted


# compiling RNN
model.compile(optimizer = SGD(learning_rate=0.01, 
                                  momentum=0.9, 
                                  nesterov=True), 
                  loss = "mean_squared_error")
 
# fitting the model
history = model.fit(X_train, Y_train, epochs = 100, verbose=1)
model.summary()


Epoch 1/100
21/21 [==============================] - 0s 2ms/step - loss: 0.0955
Epoch 2/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0548
Epoch 3/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 4/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0529
Epoch 5/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0526
Epoch 6/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0525
Epoch 7/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0528
Epoch 8/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0522
Epoch 9/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0520
Epoch 10/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0526
Epoch 11/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0520
Epoch 12/100
21/21 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 13/100
21/21 [=================